# FAIRFluids Document Creation

This notebook demonstrates how to create a FAIRFluidsDocument using the Pydantic XML models defined in `lib.py`.

The FAIRFluidsDocument is a structured way to represent fluid data with metadata, compounds, and experimental measurements.

In [14]:
# Import the FAIRFluids models
import sys
sys.path.append('/home/sga/Code/FAIRFluids')

from FAIRFluids.core.lib import (
    FAIRFluidsDocument, Version, Citation, Author, Compound, 
    Fluid, Property, Property_Group, Variable, Constraint, 
    NumValue, PropertyValue, VariableValue, ConstraintVariableType,
    eTemperature, ePressure, eComponentComposition
)

## 1. Create a Basic FAIRFluidsDocument

Let's start by creating a simple FAIRFluidsDocument with version information.

In [15]:
# Create version information
version = Version(versionMajor=1, versionMinor=0)
print(f"Version: {version.versionMajor}.{version.versionMinor}")

Version: 1.0


In [16]:
# Create a basic FAIRFluidsDocument
document = FAIRFluidsDocument(version=version)
print(f"Document created with {len(document.compound)} compounds and {len(document.fluid)} fluids")

Document created with 0 compounds and 0 fluids


## 2. Add Citation Information

Let's add citation information to the document.

In [17]:
# Create authors
author1 = Author(given_name="John", family_name="Doe")
author2 = Author(given_name="Jane", family_name="Smith")

# Create citation
citation = Citation(
    Type="journal",
    author=[author1, author2]
)

# Add citation to document
document.citation = citation

print(f"Citation added with {len(document.citation.author)} authors")
for author in document.citation.author:
    print(f"  - {author.given_name} {author.family_name}")

Citation added with 2 authors
  - John Doe
  - Jane Smith


## 3. Add Compounds

Let's add some compounds to the document. We'll use the helper method `add_to_compound`.

In [18]:
# Add water compound
water = document.add_to_compound(
    pubChemID=962,
    compund_identifier="H2O_001",
    commonName="Water",
    name_IUPAC="oxidane",
    standard_InChI="InChI=1S/H2O/h1H2",
    standard_InChI_key="XLYOFNOQVPJJNP-UHFFFAOYSA-N"
)

# Add methanol compound
methanol = document.add_to_compound(
    pubChemID=887,
    compund_identifier="CH3OH_001",
    commonName="Methanol",
    name_IUPAC="methanol",
    standard_InChI="InChI=1S/CH4O/c1-2/h2H,1H3",
    standard_InChI_key="OKKJLVBELUTLKV-UHFFFAOYSA-N"
)

print(f"Added {len(document.compound)} compounds:")
for compound in document.compound:
    print(f"  - {compound.commonName} (ID: {compound.compund_identifier})")

Added 2 compounds:
  - Water (ID: H2O_001)
  - Methanol (ID: CH3OH_001)


## 4. Add Fluid with Property and Variable Information

Now let's add a fluid with property and variable information using the helper method `add_to_fluid`.

In [19]:
# Create property group
property_group = Property_Group(
    group="volumetricProp",
    method="experimental",
    property_name="Mass Density, kg/m3"
)

# Create property
property_obj = Property(
    propertyID="DENSITY_001",
    property_group=property_group
)

# Create variable (temperature)
variable = Variable(
    variableID="TEMP_001",
    variableType=ConstraintVariableType(e_temperature=eTemperature.TEMPERATURE_K),
    variableName="Temperature"
)

# Create constraint (pressure)
constraint = Constraint(
    constraint_type=ConstraintVariableType(e_pressure=ePressure.PRESSURE_KPA),
    constraint_value=101.325,
    constraint_digits=3
)

# Create numerical values
property_value = PropertyValue(
    propValue=997.0,
    propDigits=3,
    uncertainty=0.1
)

variable_value = VariableValue(
    varValue=298.15,
    varDigits=2
)

num_value = NumValue(
    propertyValue=property_value,
    variableValue=variable_value
)

# Add fluid to document
fluid = document.add_to_fluid(
    components=["H2O_001"],  # Reference to water compound
    source_doi="10.1000/example.doi",
    property=property_obj,
    variable=variable,
    constraint=[constraint],
    num_value=num_value
)

print(f"Added fluid with {len(document.fluid)} fluids")
print(f"Fluid has {len(fluid.constraint)} constraints and numerical values")

Added fluid with 1 fluids
Fluid has 1 constraints and numerical values


## 5. Add a Binary Mixture Fluid

Let's add another fluid representing a binary mixture of water and methanol.

In [20]:
# Create property for mixture
mixture_property_group = Property_Group(
    group="volumetricProp",
    method="experimental",
    property_name="Mass Density, kg/m3"
)

mixture_property = Property(
    propertyID="DENSITY_MIX_001",
    property_group=mixture_property_group
)

# Create variable for composition
composition_variable = Variable(
    variableID="COMP_001",
    variableType=ConstraintVariableType(e_component_composition=eComponentComposition.MOLE_FRACTION),
    variableName="Mole Fraction of Methanol",
    componentID=1  # Reference to methanol
)

# Create temperature variable
temp_variable = Variable(
    variableID="TEMP_MIX_001",
    variableType=ConstraintVariableType(e_temperature=eTemperature.TEMPERATURE_K),
    variableName="Temperature"
)

# Create constraints
pressure_constraint = Constraint(
    constraint_type=ConstraintVariableType(e_pressure=ePressure.PRESSURE_KPA),
    constraint_value=101.325,
    constraint_digits=3
)

# Create numerical values for mixture
mixture_property_value = PropertyValue(
    propValue=950.5,
    propDigits=3,
    uncertainty=0.2
)

composition_value = VariableValue(
    varValue=0.25,  # 25% methanol
    varDigits=2
)

temp_value = VariableValue(
    varValue=298.15,
    varDigits=2
)

mixture_num_value = NumValue(
    propertyValue=mixture_property_value,
    variableValue=composition_value  # Note: Only one variable value can be set
)

# Add mixture fluid
mixture_fluid = document.add_to_fluid(
    components=["H2O_001", "CH3OH_001"],  # Water and methanol
    source_doi="10.1000/mixture.example.doi",
    property=mixture_property,
    variable=composition_variable,
    constraint=[pressure_constraint],
    num_value=mixture_num_value
)

print(f"Added binary mixture fluid. Total fluids: {len(document.fluid)}")

Added binary mixture fluid. Total fluids: 2


## 6. Display Document Summary

Let's see a summary of what we've created.

In [21]:
print("=== FAIRFluids Document Summary ===")
print(f"Version: {document.version.versionMajor}.{document.version.versionMinor}")
print(f"Citation Type: {document.citation.Type}")
print(f"Number of Authors: {len(document.citation.author)}")
print(f"Number of Compounds: {len(document.compound)}")
print(f"Number of Fluids: {len(document.fluid)}")

print("\n=== Compounds ===")
for i, compound in enumerate(document.compound, 1):
    print(f"{i}. {compound.commonName} ({compound.compund_identifier})")
    print(f"   PubChem ID: {compound.pubChemID}")
    print(f"   IUPAC Name: {compound.name_IUPAC}")

print("\n=== Fluids ===")
for i, fluid in enumerate(document.fluid, 1):
    print(f"{i}. Components: {fluid.components}")
    print(f"   Property: {fluid.property.property_group.property_name}")
    print(f"   Variable: {fluid.variable.variableName}")
    print(f"   Constraints: {len(fluid.constraint)}")
    if fluid.num_value and fluid.num_value.propertyValue:
        print(f"   Property Value: {fluid.num_value.propertyValue.propValue} ± {fluid.num_value.propertyValue.uncertainty}")

=== FAIRFluids Document Summary ===
Version: 1.0
Citation Type: journal
Number of Authors: 2
Number of Compounds: 2
Number of Fluids: 2

=== Compounds ===
1. Water (H2O_001)
   PubChem ID: 962
   IUPAC Name: oxidane
2. Methanol (CH3OH_001)
   PubChem ID: 887
   IUPAC Name: methanol

=== Fluids ===
1. Components: ['H2O_001']
   Property: Mass Density, kg/m3
   Variable: Temperature
   Constraints: 1
   Property Value: 997.0 ± 0.1
2. Components: ['H2O_001', 'CH3OH_001']
   Property: Mass Density, kg/m3
   Variable: Mole Fraction of Methanol
   Constraints: 1
   Property Value: 950.5 ± 0.2


## 7. Export to XML

Finally, let's export our document to XML format.

In [22]:
model.dump_json()

NameError: name 'model' is not defined

In [20]:
# Save to file
with open('fairfluids_document.xml', 'w') as f:
    f.write(xml_string)

print("XML document saved to 'fairfluids_document.xml'")

NameError: name 'xml_string' is not defined

## 8. Validation and Error Handling

Let's demonstrate some validation features and error handling.

In [20]:
# Try to create an invalid document (missing required fields)
try:
    # This should fail because Author requires given_name and family_name
    invalid_author = Author()
except Exception as e:
    print(f"Validation error caught: {e}")

# Create a valid author
valid_author = Author(given_name="Test", family_name="User")
print(f"Valid author created: {valid_author.given_name} {valid_author.family_name}")

Validation error caught: 2 validation errors for Author
given_name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
family_name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
Valid author created: Test User


## Summary

This notebook demonstrates:

1. **Importing the FAIRFluids models** from `specifications.lib`
2. **Creating a FAIRFluidsDocument** with version information
3. **Adding citation information** with authors
4. **Adding compounds** with metadata (PubChem ID, IUPAC names, etc.)
5. **Creating fluids** with properties, variables, and constraints
6. **Adding numerical values** for measurements
7. **Exporting to XML** format
8. **Validation** and error handling

The FAIRFluidsDocument provides a structured way to represent fluid data that is:
- **Findable**: Well-organized metadata
- **Accessible**: Standard XML format
- **Interoperable**: Consistent schema
- **Reusable**: Rich metadata and provenance information